In [0]:
import numpy as np
import torch
from sklearn.metrics import roc_auc_score

import logging
import time

In [0]:
def test(testloader, classifier, c=None, R=None, objective='one-class', device='cuda'):
    logger = logging.getLogger()

    # Set device for network
    classifier = classifier.to(device)

    # Testing
    logger.info('Starting testing...')
    start_time = time.time()
    idx_label_score = []
    classifier.eval()
    with torch.no_grad():
        for data in testloader:
            inputs, labels, idx = data
            inputs = inputs.to(device)
            outputs = classifier(inputs)
            dist = torch.sum((outputs - c) ** 2, dim=1)
            if objective == 'soft-boundary':
                scores = dist - R ** 2
            else:
                scores = dist

            # Save triples of (idx, label, score) in a list
            idx_label_score += list(zip(idx.cpu().data.numpy().tolist(),
                                        labels.cpu().data.numpy().tolist(),
                                        scores.cpu().data.numpy().tolist()))

    test_time = time.time() - start_time
    logger.info('Testing time: %.3f' % test_time)

    test_scores = idx_label_score

    # Compute AUC
    _, labels, scores = zip(*idx_label_score)
    labels = np.array(labels)
    scores = np.array(scores)

    test_auc = roc_auc_score(labels, scores)
    logger.info('Test set AUC: {:.2f}%'.format(100. * test_auc))

    logger.info('Finished testing.')